In [1]:
from IPython.display import display, Math, Latex
display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))

<IPython.core.display.Math object>

# Loss Functions
In this article, we will focus on the selection of loss functions for regression and classification problems. First, we justify minimizing the mean square error (MSE) as a loss function for linear regression. We will proceed from the statement of the regression as a task with an infinite number of outcomes, thereby naturally assuming that the loss function, in this case, will be continuous. In contrast, the problem of classification has a discrete number of outcomes, and its loss function does not have the same nature as for regression. We will introduce a metric on the distribution space approximating the classification values and show that although the introduced value may not have all the properties of the metric, it can nevertheless serve to determine the “distance” between distributions, that is, it can be successfully used as a loss function. The considered quantity called cross-entropy, and it is widely used in commercial libraries (TensorFlow, PyTorch) when constructing classification models.

## Maximum likelihood estimation (MLE) and KL Divergence
There are many ways to introduce a metric on a distribution space. Some metrics were borrowed from functional analysis, while others, due to their special properties, were introduced for special cases. Such cases include pre-metrics that satisfy only part of the axiomatics of metrics, however, they are often used to specify the topology of the distribution space, and to some extent play the role of the distance on it. Such is the pre-metric that is known from information theory: the Kullback-Leibler divergence. For discrete distributions, it is defined as


$$D_{KL}(P||Q)  =\sum_{x \in X} P(x) \log(\frac{P(x)}{Q(x)})$$

